In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [2]:
data = pd.read_csv('dataset.csv').drop(['Unnamed: 0'],axis=1)

In [3]:
data = data.sample(frac=1).reset_index(drop=True)

In [4]:
train_data = data.head(int(225*0.8))

In [5]:
test_data = data.tail(int(225*0.2))

In [6]:
display(train_data)
display(test_data)

,Label,Tweet
0,1,"RT @nytimesworld: More than 2,500 medical work..."
1,0,@DeZurdaTeam @DiazCanelB @marti160patria @SecU...
2,0,RT @alfonslopeztena: Hundreds of Hong Kong med...
3,0,RT @frontlinepbs: As the number of cases of th...
4,0,@CNN infectious diseases typically look more s...
...,...,...
175,0,RT @ChannelNewsAsia: Hong Kong medical workers...
176,1,RT @alfonslopeztena: Hundreds of Hong Kong med...
177,0,RT @frontlinepbs: As the number of cases of th...
178,1,"RT @globalnews: More than 2,000 hospital worke..."


,Label,Tweet
181,0,@caijingxiang Best way for coronavirus prevent...
182,0,RT @annafifield: A person has died of coronavi...
183,0,RT @sehof: The censorship instructions with re...
184,0,RT @nytimes: In the middle of the coronavirus ...
185,1,RT @YohendrisQ: @DeZurdaTeam @DiazCanelB @mart...
186,0,"RT @AFP: Hong Kong medical workers strike, dem..."
187,0,RT @ReutersChina: Hong Kong reports first coro...
188,0,RT @balajis: The #coronavirus has now killed m...
189,0,"RT @QAnonNotables: Dr. Francis Boyle, drafter ..."
190,1,"RT @AFP: Hong Kong medical workers strike, dem..."


In [7]:
train_data = pd.DataFrame({
    'id':range(len(train_data)),
    'label': train_data['Label'],
    'alpha': ['a']*train_data.shape[0],
    'text': train_data['Tweet'].replace(r'\n', ' ', regex=True)
})

train_data.head()

,id,label,alpha,text
0,0,1,a,"RT @nytimesworld: More than 2,500 medical work..."
1,1,0,a,@DeZurdaTeam @DiazCanelB @marti160patria @SecU...
2,2,0,a,RT @alfonslopeztena: Hundreds of Hong Kong med...
3,3,0,a,RT @frontlinepbs: As the number of cases of th...
4,4,0,a,@CNN infectious diseases typically look more s...


In [8]:
test_data = pd.DataFrame({
    'id': range(len(test_data)),
    'label': test_data['Label'],
    'alpha': ['a']*test_data.shape[0],
    'text': test_data['Tweet'].replace(r'\n', ' ', regex=True)
})

test_data.head()

,id,label,alpha,text
181,0,0,a,@caijingxiang Best way for coronavirus prevent...
182,1,0,a,RT @annafifield: A person has died of coronavi...
183,2,0,a,RT @sehof: The censorship instructions with re...
184,3,0,a,RT @nytimes: In the middle of the coronavirus ...
185,4,1,a,RT @YohendrisQ: @DeZurdaTeam @DiazCanelB @mart...


In [9]:
from utilstransformers import *

In [10]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange

from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
args = {
    'data_dir': 'data/',
    'model_type':  'xlnet',
    'model_name': 'xlnet-base-cased',
    'task_name': 'binary',
    'output_dir': 'outputs/',
    'cache_dir': 'cache/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 8,
    'eval_batch_size': 8,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 1,
    'weight_decay': 0,
    'learning_rate': 4e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 2000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': True,
    'reprocess_input_data': True,
    'notes': 'Using  '
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [11]:
if args['do_train']:
    train_dataset = load_and_cache_examples(task, tokenizer)
    global_step, tr_loss = train(train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

NameError: name 'args' is not defined